---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 
          'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 
          'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 
          'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 
          'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 
          'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 
          'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 
          'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 
          'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 
          'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 
          'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 
          'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 
          'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 
          'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 
          'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 
          'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove 
       newline character '\n'. '''
    
    # read in txt file and append to a list
    txt_list = []
    with open('university_towns.txt', 'r') as file:
        for line in file:
            txt_list.append(line.replace('\n','').split(' ('))

    # convert to dataframe
    txt_DataFrame = pd.DataFrame(txt_list)

    # rename column
    txt_DataFrame.columns = ['raw_state_region', 'raw_uni', 'State', 'RegionName']

    # re-populate State column
    txt_DataFrame.State = None
    txt_DataFrame.State = txt_DataFrame\
            .raw_state_region\
            .apply(lambda x: None if x.find('[edit]') == -1 else x.replace('[edit]',''))

    # re-populate RegionName column
    txt_DataFrame.RegionName = None
    txt_DataFrame.RegionName = txt_DataFrame\
                    .raw_state_region\
                    .apply(lambda x: None if x.find('[edit]') != -1 else x)

    # drop unncecessary columns
    txt_DataFrame.drop(['raw_state_region', 'raw_uni'], axis=1, inplace=True)

    # fill State column
    txt_DataFrame = txt_DataFrame.fillna(method='ffill')

    # drop None in RegionName column
    txt_DataFrame.dropna(inplace=True)
    """
    # final clean up
    txt_DataFrame.RegionName = txt_DataFrame.RegionName.str\
                            .replace(', South Central College', '')
    txt_DataFrame.RegionName = txt_DataFrame.RegionName.str\
                            .replace(', also known as "Templetown"', '')
    txt_DataFrame = txt_DataFrame[~txt_DataFrame.RegionName.str.contains(':')]
    """
    return txt_DataFrame

get_list_of_university_towns().head()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # read excel file
    gdp = pd.read_excel('gdplev.xls', usecols=[4,6], skiprows=4)\
            .dropna()
    
    # rename column
    gdp.columns = ['Quarter', 'GDP_chained_2009']

    # shift GDP column one down
    gdp['Lag1'] = gdp.GDP_chained_2009.shift(1)
    
    # filter: 2000Q1 and above
    gdp = gdp.loc[gdp.Quarter >= '2000q1'].reset_index(drop=True)
    
    # indicator for decline in GDP
    gdp['decline'] = gdp\
        .apply(lambda x: 1 if (x['GDP_chained_2009'] < x['Lag1']) else 0, axis=1)
    
    # find the start
    start = gdp[(gdp.decline == 1) & # current quarter: decline
                (gdp.decline.shift(1) == 0) & # last quarter: growth
                (gdp.decline.shift(-1) == 1)]['Quarter'].values[0] # next quater: decline
    
    return start

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    # read excel file
    gdp = pd.read_excel('gdplev.xls', usecols=[4,6], skiprows=4).dropna()
    
    # rename column
    gdp.columns = ['Quarter', 'GDP_chained_2009']

    # shift GDP column one down
    gdp['Lag1'] = gdp.GDP_chained_2009.shift(1)
    
    # filter: 2000Q1 and above
    gdp = gdp.loc[gdp.Quarter >= '2000q1'].reset_index(drop=True)
    
    # indicator for decline in GDP
    gdp['decline'] = gdp\
        .apply(lambda x: 1 if (x['GDP_chained_2009'] < x['Lag1']) else 0, axis=1)
    
    # find the start
    end = gdp[(gdp.decline == 0) & # current quarter: growth
                (gdp.decline.shift(1) == 0) & # last quarter: growth
                (gdp.decline.shift(2) == 1) & # second last quarter: decline
                (gdp.decline.shift(3) == 1) & # third last quarter: decline
                (gdp.decline.shift(-1) == 0)]['Quarter'].values[0] # next quarter: growth
       
    return end

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # read excel file
    gdp = pd.read_excel('gdplev.xls', usecols=[4,6], skiprows=4).dropna()

    # rename column
    gdp.columns = ['Quarter', 'GDP_chained_2009']

    # shift GDP column one down
    gdp['Lag1'] = gdp.GDP_chained_2009.shift(1)

    # filter: 2000Q1 and above
    gdp = gdp.loc[gdp.Quarter >= '2000q1']\
        .reset_index(drop=True)

    # indicator for decline in GDP
    gdp['decline'] = gdp\
        .apply(lambda x: 1 if (x['GDP_chained_2009'] < x['Lag1']) else 0, axis=1)
    
    # get recession start and end
    start = get_recession_start()
    end = get_recession_end()
    
    # min value of chained gdp with start and end
    bottom = gdp.loc[np.logical_and(gdp.Quarter >= start, gdp.Quarter <= end)]
    bottom = bottom.set_index('Quarter')
    bottom = bottom.GDP_chained_2009.idxmin()
    
    return bottom

get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    # read data and replace state name abbreviations
    data = pd.read_csv('City_Zhvi_AllHomes.csv')
    data.State = data.State.apply(lambda x: states[x])
    
    # set index
    data = data.sort_values(['State', 'RegionName']).set_index(['State', 'RegionName'])
    
    # month columns to keep
    month_columns = [str(y) + '-' + str(m) if len(str(m)) == 2 
                     else str(y) + '-0' + str(m) 
                     for y in np.arange(2000, 2017) for m in np.arange(1,13)][:-4]
    data = data[month_columns]
    
    # change column names so that months are replaced with quarters
    # q1: 1,2,3
    # q2: 4,5,6
    # q3: 7,8,9
    # q4: 10,11,12
    
    month_columns = [m.replace('-01', 'q1').replace('-02', 'q1').replace('-03', 'q1') 
                     for m in month_columns]
    month_columns = [m.replace('-04', 'q2').replace('-05', 'q2').replace('-06', 'q2') 
                     for m in month_columns]
    month_columns = [m.replace('-07', 'q3').replace('-08', 'q3').replace('-09', 'q3') 
                     for m in month_columns]
    month_columns = [m.replace('-10', 'q4').replace('-11', 'q4').replace('-12', 'q4') 
                     for m in month_columns]
    
    # rename columns
    data.columns = month_columns

    # create a empty data frame with the same index
    dataAgg = pd.DataFrame(index=data.index)
    
    # loop over all quaters, calculate mean across columns and add to
    # the empty dataframe
    for q in np.unique(month_columns):
        dataAgg[q] = pd.DataFrame(data[q].mean(axis=1))
        
    return dataAgg  

convert_housing_data_to_quarters().head()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   

                            2002q2    ...            2014q2         2014q3  \
State   RegionName                    ...                                    
Alabama Adamsville    73133.333333    ...      77066.666667   75966.666667   
        Alabaster    128000.000000    ...     147133.333333  147633.333333   
        Albertville   76366.666667    ...      84033.333333   84766.666667   
        Arab          87700.000000    ...     113366.666667  111700.000000   
        Ardmore                NaN    ...     140533.333333  139566.666667   

                       2014q4         2015q1         2015q2         2015q3  \
State   RegionName                                                           
Alabama Adamsville    71900.0   71666.666667   73033.333333   73933.333333   
        Alabaster    148700.0  148900.000000  149566.666667  150366.666667   
        Albertville   86800.0   88466.666667   89500.000000   90233.333333   
        Arab         111600.0  110166.666667  109433.333333  110900.000000   
        Ardmore      140900.0  143233.333333  143000.000000  144600.000000   

                            2015q4         2016q1         2016q2    2016q3  
State   RegionName                                                          
Alabama Adamsville    73866.666667   74166.666667   74933.333333   74700.0  
        Alabaster    151733.333333  153466.666667  155100.000000  155850.0  
        Albertville   91366.666667   92000.000000   92466.666667   92200.0  
        Arab         112233.333333  110033.333333  110100.000000  112000.0  
        Ardmore      143966.666667  142566.666667  143233.333333  141950.0  

[5 rows x 67 columns]

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    uniTowns = get_list_of_university_towns().set_index(['State', 'RegionName'])
    start = get_recession_start()
    end = get_recession_end()
    bottom = get_recession_bottom()
    homePrice = convert_housing_data_to_quarters()
    recessionYears = homePrice.loc[:,start:bottom]
    recessionYears['Change'] = homePrice[start] - homePrice[bottom]

    priceChangeUniTowns = pd.merge(uniTowns, recessionYears, 
                                   right_index=True, left_index=True, how='left')\
                            .dropna()\
                            .Change
    priceChangeNonUniTowns = pd.merge(uniTowns, recessionYears, 
                                      right_index=True, left_index=True, how='right')\
                            .dropna()\
                            .Change

    test = ttest_ind(priceChangeUniTowns, priceChangeNonUniTowns)
    different = test[1] < 0.01
    p = test[1]
     
    if np.mean(priceChangeUniTowns) < np.mean(priceChangeNonUniTowns):
        better = 'university town'
    else:
        better = 'non-university town'
    
    return (different, p, better)

run_ttest()

(True, 0.0051203879027585881, 'university town')